In [26]:
from collections import defaultdict

import codecs
import gensim
import networkx as nx
import numpy as np

from bs4 import BeautifulSoup

In [11]:
G = nx.read_gexf("datasets/influences.gexf")
names = [ n for n in G.nodes() ]

In [85]:
def philosopher2vec(name):
  def get_text(name):
    txt = codecs.open("datasets/pages/%s.html" % name,"r", "utf-8-sig").read()

    txt = BeautifulSoup(txt, "html.parser")
    txt = txt.get_text()

    return txt
  
  def tokenize(sentences):
    stoplist = set(codecs.open("datasets/stopwords.txt","r", "utf-8-sig").read().split())
    texts = [[word.replace(",", "") for word in document.lower().split() if word not in stoplist] for document in sentences]

    
    frequency = defaultdict(int)

    for text in texts:
      for token in text:
        frequency[token] += 1
    
    texts = [[token for token in text if frequency[token] > 1] for text in texts]
    
    return texts
  
  def to_dict(tokens):
    d = gensim.corpora.Dictionary(tokens)

    return d
  
  def to_corpus(d, t):
    c = [d.doc2bow(text) for text in t]
    
    return c

  
  txt = get_text(name)
  sentences = txt.split(".")
  
  tokens = tokenize(sentences)
  dictionary = to_dict(tokens)
  
  corpus = to_corpus(dictionary, tokens)

  bigrams = gensim.models.Phrases(tokens)

  lsi = gensim.models.lsimodel.LsiModel(corpus, id2word=dictionary, num_topics=300)
  word2vec = gensim.models.Word2Vec(bigrams[tokens], size=100, window=5, min_count=5, workers=4)

  # print tokens
  
  print name
  print len(sentences)
  print np.sum([ len(t) for t in tokens ])
  print dictionary
  # print bigrams
  # print lsi.print_topics(10)
  print
  
  for (u,s) in word2vec.most_similar(positive=['work']):
    print "%s: %s" % (u,s)
    
  print

#names = names[0:10]
names = ["Karl_Popper", "Martin_Heidegger", "Ludwig_Wittgenstein"]
# names = ["Ludwig_Wittgenstein"]
  
[ philosopher2vec(name) for name in names ];

Karl_Popper
923
6881
Dictionary(1441 unique tokens: [u'', u'searle', u'writings', u'four', u'whose']...)

philosophy: 0.590414404869
theory: 0.568217277527
history: 0.558463215828
scientific: 0.533928751945
growth: 0.515239536762
popper's: 0.506954073906
popper: 0.501690924168
science: 0.500450849533
der: 0.499593257904
evolution: 0.497165858746

Martin_Heidegger


1007
8264
Dictionary(1568 unique tokens: [u'', u'limited', u'writings', u"lacoue-labarthe's", u'bedeutungslehre']...)

heidegger: 0.952893555164
heidegger's: 0.952154695988
philosophy: 0.948971688747
works: 0.9304292202
history: 0.929019749165
^: 0.92490041256
being: 0.924608290195
"the: 0.921348214149
heidegger:: 0.91824901104
all: 0.918234944344

Ludwig_Wittgenstein
1710
10419
Dictionary(1924 unique tokens: [u'', u'searle', u'writings', u'augustine', u'linz[edit]']...)

wittgenstein: 0.860655367374
philosophy: 0.855276286602
theory: 0.841419219971
said: 0.83538967371
philosophical: 0.819308996201
language: 0.814856648445
wittgenstein's: 0.805397212505
logical: 0.796191871166
remarks: 0.786036014557
john: 0.785959005356

